In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

# **Fine-tune the flan-t5-small model**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset


zain bhae run above and below the cell. upload the dataset in colab then turn on the gpu

In [ ]:
# Load the dataset
df = pd.read_excel('/content/Query Dataset (multi-ques).xlsx')

# Preprocess the dataset
df['Ayah'] = df['Ayah'].astype(str)
df = df.drop(columns=['Unnamed: 6', 'Topic', 'Keywords'], errors='ignore')
df.columns = df.columns.str.strip()
df = df.dropna()
df['target_text'] = df.apply(lambda x: f"Surah {x['Surah']}, Ayah {x['Ayah']}: {x['Quranic Verse']}", axis=1)
df = df[['Query', 'target_text']]
df = df.rename(columns={'Query': 'input_text'})

# Split dataset
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert to Hugging Face dataset format
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

# Tokenization function
def preprocess_function(examples):
    inputs = [f"Query: {query}" for query in examples['input_text']]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['target_text'], max_length=256, truncation=True, padding="max_length")
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply preprocessing
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./flan-t5-quran",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=25,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=500,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none",
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()

# Save fine-tuned model
model.save_pretrained("./flan-t5-quran-finetuned")
tokenizer.save_pretrained("./flan-t5-quran-finetuned")

# Load model for inference
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Function to generate responses
def generate_response(query, model, tokenizer, device="cuda"):
    input_text = f"Query: {query}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=256, truncation=True).to(device)
    outputs = model.generate(input_ids, max_length=256, num_beams=5, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example query
query = "What does the Quran say about patience?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/404 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-3-aef1c79419c2>:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,1.923600,1.366023
2,0.418000,0.311237
3,0.286000,0.238395
4,0.239700,0.211582
5,0.244400,0.194485
6,0.217800,0.182894
7,0.236100,0.174957
8,0.233400,0.169844
9,0.207400,0.166734
10,0.179800,0.165711


NameError: name 'torch' is not defined

In [ ]:
import torch

In [ ]:
# Load model for inference
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Function to generate responses
def generate_response(query, model, tokenizer, device="cuda"):
    input_text = f"Query: {query}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=256, truncation=True).to(device)
    outputs = model.generate(input_ids, max_length=256, num_beams=5, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example query
query = "WWhat does the Quran say about helping others?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")
# Example query
query = "What does the Quran say about charity?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")
# Example query
query = "What guidance does the Quran give about arrogance?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")
# Example query
query = "What does the Quran say about forgiveness?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")
# Example query
query = "How can a disbeliever believe after Allah has cursed him and expelled him from His mercy?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")

Query: WWhat does the Quran say about helping others?
Response: Surah 2, Ayah 153: And do not kill yourselves. Indeed, Allah is with you ever Merciful.


In [ ]:
# Example query
query = "What does the Quran say about charity?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")

Query: What does the Quran say about charity?
Response: Surah 2, Ayah 261: And do not turn your cheek in disdain toward people.


In [ ]:
# Example query
query = "What guidance does the Quran give about arrogance?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")

Query: What guidance does the Quran give about arrogance?
Response: Surah 2, Ayah 153: And do not turn your cheek in arrogance nor walk in fear of Allah.


In [ ]:
# Example query
query = "What does the Quran say about forgiveness?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")

Query: What does the Quran say about forgiveness?
Response: Surah 64, Ayah 14: And if you pardon and overlook and forgive – then indeed, Allah is Forgiving and Merciful.


In [ ]:
# Example query
query = "How can a disbeliever believe after Allah has cursed him and expelled him from His mercy?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")

Query: How can a disbeliever believe after Allah has cursed him and expelled him from His mercy?
Response: Surah 94, Ayah 7: And We have enjoined upon man [care] for his sins.


In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Trainer,
    TrainingArguments
)
from datasets import Dataset

# Load the dataset
df = pd.read_excel('/content/Query Dataset.xlsx')

# Convert Ayah to string (if it's not already)
df['Ayah'] = df['Ayah'].astype(str)

# Drop unnecessary columns
df = df.drop(columns=['Unnamed: 6', 'Topic', 'Keywords'], errors='ignore')

# Remove leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Ensure no missing values
df = df.dropna()

# Ensure correct formatting for target text
df['target_text'] = df.apply(lambda x: f"Surah {x['Surah']}, Ayah {x['Ayah']}: {x['Quranic Verse']}", axis=1)

# Keep only required columns
df = df[['Query', 'target_text']]
df = df.rename(columns={'Query': 'input_text'})

# Split dataset
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert to Hugging Face dataset format
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")

# Adjust Tokenization Process
def preprocess_function(examples):
    inputs = [f"Query: {query}" for query in examples['input_text']]

    model_inputs = tokenizer(
        inputs,
        max_length=256,
        truncation=True,
        padding="max_length",
    )

    # Tokenizing targets correctly
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['target_text'],
            max_length=256,
            truncation=True,
            padding="max_length",
        )

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply preprocessing
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

# Load model
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

# Training arguments
training_args = TrainingArguments(
    output_dir="./flan-t5-quran",
    evaluation_strategy="epoch",
    learning_rate=1e-4,  # Increased learning rate
    per_device_train_batch_size=16,  # Increased batch size
    per_device_eval_batch_size=16,
    num_train_epochs=8,  # Increased epochs for better learning
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=500,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none",
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()

# Save fine-tuned model
model.save_pretrained("./flan-t5-quran-finetuned")
tokenizer.save_pretrained("./flan-t5-quran-finetuned")

# Load model for inference
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Function to generate responses
def generate_response(query, model, tokenizer, device="cuda"):
    input_text = f"Query: {query}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=256, truncation=True).to(device)

    outputs = model.generate(
        input_ids,
        max_length=256,
        num_beams=5,
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example query
query = "What does the Quran say about patience?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")


Map:   0%|          | 0/727 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/81 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-6-3c12d9e67796>:91: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,6.224000,4.559031
2,3.195900,2.473404
3,1.867500,1.063887
4,0.973100,0.548174
5,0.609300,0.384113
6,0.468000,0.325087
7,0.423400,0.304793
8,0.402300,0.299305


Query: What does the Quran say about patience?
Response: Surah 2, Ayah 63: And if you are patient, you will be patient.


**Flan-T5 Small was giving results like:
Query: What does the Quran say about patience?
Response: Surah 2, Ayah 63: And if you are patient, you will be patient ****.

.** **Because of this, I had to switch to Flan-T5 Base, but even this is giving only 90% accurate results**

# **Fine tune the flan-t5-base model**

This code fine-tunes the Flan-T5 base model on a Quranic dataset to generate Surah,Ayah and verses based on a query. It preprocesses the dataset, tokenizes inputs, and trains the model using Hugging Face's Trainer. After fine-tuning, it saves the model and provides an inference function to generate Quranic references for given queries

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Trainer,
    TrainingArguments
)
from datasets import Dataset

# Load the dataset
df = pd.read_excel('/content/Query Dataset (multi-ques).xlsx')

# Convert Ayah to string (if it's not already)
df['Ayah'] = df['Ayah'].astype(str)

# Drop unnecessary columns
df = df.drop(columns=['Unnamed: 6', 'Topic', 'Keywords'], errors='ignore')

# Remove leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Ensure no missing values
df = df.dropna()

# Ensure correct formatting for target text
df['target_text'] = df.apply(lambda x: f"Surah {x['Surah']}, Ayah {x['Ayah']}: {x['Quranic Verse']}", axis=1)

# Keep only required columns
df = df[['Query', 'target_text']]
df = df.rename(columns={'Query': 'input_text'})

# Split dataset
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert to Hugging Face dataset format
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

# Adjust Tokenization Process
def preprocess_function(examples):
    inputs = [f"Query: {query}" for query in examples['input_text']]

    model_inputs = tokenizer(
        inputs,
        max_length=256,
        truncation=True,
        padding="max_length",
    )

    # Tokenizing targets correctly
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['target_text'],
            max_length=256,
            truncation=True,
            padding="max_length",
        )

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply preprocessing
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

# Load model
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

# Training arguments
training_args = TrainingArguments(
    output_dir="./flan-t5-quran-base",
    evaluation_strategy="epoch",
    learning_rate=1e-4,  # Increased learning rate
    per_device_train_batch_size=8,  # Adjust batch size based on available memory
    per_device_eval_batch_size=8,
    num_train_epochs=25,  # Increased epochs for better learning
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=500,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none",
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()

# Save fine-tuned model
model.save_pretrained("./flan-t5-quran-base-finetuned")
tokenizer.save_pretrained("./flan-t5-quran-base-finetuned")

# Load model for inference
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Function to generate responses
def generate_response(query, model, tokenizer, device="cuda"):
    input_text = f"Query: {query}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=256, truncation=True).to(device)

    outputs = model.generate(
        input_ids,
        max_length=256,
        num_beams=5,
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example query
query = "What does the Quran say about patience?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-3-59b07b9dbd99>:91: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.129600,0.107982
2,0.086200,0.068147
3,0.065600,0.046928
4,0.050700,0.035422
5,0.046900,0.028401
6,0.037800,0.023678
7,0.038500,0.021090
8,0.040000,0.020329


Query: What does the Quran say about patience?
Response: Surah 2, Ayah 153: Indeed, Allah is with the patient.


In [ ]:
# Example query
query = "What does the Quran say about patience?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")

In [ ]:
# Example query
query = "How can a disbeliever believe after Allah has cursed him and expelled him from His mercy?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")

Query: How can a disbeliever believe after Allah has cursed him and expelled him from His mercy?
Response: Surah 39, Ayah 53: Say, ‘O My servants who have harmed yourselves by your own actions, do not despair of the mercy of Allah.


In [ ]:
# Example query
query = "How can a disbeliever believe after Allah has cursed him and expelled him from His mercy?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")

Query: How can a disbeliever believe after Allah has cursed him and expelled him from His mercy?
Response: Surah 39, Ayah 53: Say, ‘O My servants who have harmed yourselves by your own actions, do not despair of the mercy of Allah.


In [ ]:
# Example query
query = "What does the Quran say about charity?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")

Query: What does the Quran say about charity?
Response: Surah 2, Ayah 261: The example of those who spend their wealth in the way of Allah is like that of a seed.


In [ ]:
# Example query
query = "What does the Quran say about helping others?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")

Query: What does the Quran say about helping others?
Response: Surah 2, Ayah 261: The example of those who spend their wealth in the way of Allah is like that of a seed.


In [ ]:
# Example query
query = "What does the Quran say about forgiveness?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")


Query: What does the Quran say about forgiveness?
Response: Surah 64, Ayah 14: And if you pardon and overlook and forgive – then indeed, Allah is Forgiving and Merciful.


In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Trainer,
    TrainingArguments
)
from datasets import Dataset

# Load the dataset
df = pd.read_excel('/content/Query Dataset (multi-ques).xlsx')

# Convert Ayah to string (if it's not already)
df['Ayah'] = df['Ayah'].astype(str)

# Drop unnecessary columns
df = df.drop(columns=['Unnamed: 6', 'Topic', 'Keywords'], errors='ignore')

# Remove leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Ensure no missing values
df = df.dropna()

# Ensure correct formatting for target text
df['target_text'] = df.apply(lambda x: f"Surah {x['Surah']}, Ayah {x['Ayah']}: {x['Quranic Verse']}", axis=1)

# Keep only required columns
df = df[['Query', 'target_text']]
df = df.rename(columns={'Query': 'input_text'})

# Split dataset
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert to Hugging Face dataset format
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

# Adjust Tokenization Process
def preprocess_function(examples):
    inputs = [f"Query: {query}" for query in examples['input_text']]

    model_inputs = tokenizer(
        inputs,
        max_length=256,
        truncation=True,
        padding="max_length",
    )

    # Tokenizing targets correctly
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['target_text'],
            max_length=256,
            truncation=True,
            padding="max_length",
        )

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply preprocessing
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

# Load model
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

# Training arguments
training_args = TrainingArguments(
    output_dir="./flan-t5-quran-base",
    evaluation_strategy="epoch",
    learning_rate=1e-4,  # Increased learning rate
    per_device_train_batch_size=8,  # Adjust batch size based on available memory
    per_device_eval_batch_size=8,
    num_train_epochs=25,  # Increased epochs for better learning
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=500,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none",
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()

# Save fine-tuned model
model.save_pretrained("./flan-t5-quran-base-finetuned")
tokenizer.save_pretrained("./flan-t5-quran-base-finetuned")

# Load model for inference
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Function to generate responses
def generate_response(query, model, tokenizer, device="cuda"):
    input_text = f"Query: {query}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=256, truncation=True).to(device)

    outputs = model.generate(
        input_ids,
        max_length=256,
        num_beams=5,
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example query
query = "What does the Quran say about patience?"
response = generate_response(query, model, tokenizer, device)
print(f"Query: {query}\nResponse: {response}")


ModuleNotFoundError: No module named 'datasets'